### Importing packages

In [1]:
import tensorflow as tf
import tensorflow_data_validation as tfdv
from tfx import v1 as tfx
from tfx.types import standard_artifacts

from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from google.protobuf.json_format import MessageToDict
from tensorflow_metadata.proto.v0 import schema_pb2

import os
import pprint

2023-08-10 13:44:39.414253: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-10 13:44:40.219951: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-10 13:44:40.222930: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-10 13:44:42.687789: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
pp = pprint.PrettyPrinter()

In [3]:
#Defining paths
_pipeline_root = './pipeline/'
_data_root = './data/census_data'
_data_filepath = os.path.join(_data_root, 'adult.data')

### Create interactive context

In [4]:
context = InteractiveContext(pipeline_root = _pipeline_root)

### ExampleGen

In [5]:
example_gen = tfx.components.CsvExampleGen(input_base = _data_root)
context.run(example_gen)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 26
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

### StatisticsGen

In [6]:
statistics_gen = tfx.components.StatisticsGen(
    examples = example_gen.outputs['examples']
)

context.run(statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 27
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

### SchemaGen

In [7]:
schema_gen = tfx.components.SchemaGen(
    statistics = statistics_gen.outputs['statistics']
)
context.run(schema_gen)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 28
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [8]:
context.show(schema_gen.outputs['schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'age',INT,required,,-
'capital-gain',INT,required,,-
'capital-loss',INT,required,,-
'education',STRING,required,,'education'
'education-num',INT,required,,-
'fnlwgt',INT,required,,-
'hours-per-week',INT,required,,-
'label',STRING,required,,'label'
'marital-status',STRING,required,,'marital-status'


,Values
Domain,
'education',"' 10th', ' 11th', ' 12th', ' 1st-4th', ' 5th-6th', ' 7th-8th', ' 9th', ' Assoc-acdm', ' Assoc-voc', ' Bachelors', ' Doctorate', ' HS-grad', ' Masters', ' Preschool', ' Prof-school', ' Some-college'"
'label',"' <=50K', ' >50K'"
'marital-status',"' Divorced', ' Married-AF-spouse', ' Married-civ-spouse', ' Married-spouse-absent', ' Never-married', ' Separated', ' Widowed'"
'native-country',"' ?', ' Cambodia', ' Canada', ' China', ' Columbia', ' Cuba', ' Dominican-Republic', ' Ecuador', ' El-Salvador', ' England', ' France', ' Germany', ' Greece', ' Guatemala', ' Haiti', ' Honduras', ' Hong', ' Hungary', ' India', ' Iran', ' Ireland', ' Italy', ' Jamaica', ' Japan', ' Laos', ' Mexico', ' Nicaragua', ' Outlying-US(Guam-USVI-etc)', ' Peru', ' Philippines', ' Poland', ' Portugal', ' Puerto-Rico', ' Scotland', ' South', ' Taiwan', ' Thailand', ' Trinadad&Tobago', ' United-States', ' Vietnam', ' Yugoslavia', ' Holand-Netherlands'"
'occupation',"' ?', ' Adm-clerical', ' Armed-Forces', ' Craft-repair', ' Exec-managerial', ' Farming-fishing', ' Handlers-cleaners', ' Machine-op-inspct', ' Other-service', ' Priv-house-serv', ' Prof-specialty', ' Protective-serv', ' Sales', ' Tech-support', ' Transport-moving'"
'race',"' Amer-Indian-Eskimo', ' Asian-Pac-Islander', ' Black', ' Other', ' White'"
'relationship',"' Husband', ' Not-in-family', ' Other-relative', ' Own-child', ' Unmarried', ' Wife'"
'sex',"' Female', ' Male'"
'workclass',"' ?', ' Federal-gov', ' Local-gov', ' Never-worked', ' Private', ' Self-emp-inc', ' Self-emp-not-inc', ' State-gov', ' Without-pay'"


### Modifying the schema

In [9]:
schema_uri = schema_gen.outputs['schema']._artifacts[0].uri
schema = tfdv.load_schema_text(os.path.join(schema_uri, 'schema.pbtxt'))

In [10]:
tfdv.set_domain(schema, 'age', schema_pb2.IntDomain(name = 'age', min = 17, max = 90))
tfdv.display_schema(schema)

,Type,Presence,Valency,Domain
Feature name,,,,
'age',INT,required,,min: 17; max: 90
'capital-gain',INT,required,,-
'capital-loss',INT,required,,-
'education',STRING,required,,'education'
'education-num',INT,required,,-
'fnlwgt',INT,required,,-
'hours-per-week',INT,required,,-
'label',STRING,required,,'label'
'marital-status',STRING,required,,'marital-status'


,Values
Domain,
'education',"' 10th', ' 11th', ' 12th', ' 1st-4th', ' 5th-6th', ' 7th-8th', ' 9th', ' Assoc-acdm', ' Assoc-voc', ' Bachelors', ' Doctorate', ' HS-grad', ' Masters', ' Preschool', ' Prof-school', ' Some-college'"
'label',"' <=50K', ' >50K'"
'marital-status',"' Divorced', ' Married-AF-spouse', ' Married-civ-spouse', ' Married-spouse-absent', ' Never-married', ' Separated', ' Widowed'"
'native-country',"' ?', ' Cambodia', ' Canada', ' China', ' Columbia', ' Cuba', ' Dominican-Republic', ' Ecuador', ' El-Salvador', ' England', ' France', ' Germany', ' Greece', ' Guatemala', ' Haiti', ' Honduras', ' Hong', ' Hungary', ' India', ' Iran', ' Ireland', ' Italy', ' Jamaica', ' Japan', ' Laos', ' Mexico', ' Nicaragua', ' Outlying-US(Guam-USVI-etc)', ' Peru', ' Philippines', ' Poland', ' Portugal', ' Puerto-Rico', ' Scotland', ' South', ' Taiwan', ' Thailand', ' Trinadad&Tobago', ' United-States', ' Vietnam', ' Yugoslavia', ' Holand-Netherlands'"
'occupation',"' ?', ' Adm-clerical', ' Armed-Forces', ' Craft-repair', ' Exec-managerial', ' Farming-fishing', ' Handlers-cleaners', ' Machine-op-inspct', ' Other-service', ' Priv-house-serv', ' Prof-specialty', ' Protective-serv', ' Sales', ' Tech-support', ' Transport-moving'"
'race',"' Amer-Indian-Eskimo', ' Asian-Pac-Islander', ' Black', ' Other', ' White'"
'relationship',"' Husband', ' Not-in-family', ' Other-relative', ' Own-child', ' Unmarried', ' Wife'"
'sex',"' Female', ' Male'"
'workclass',"' ?', ' Federal-gov', ' Local-gov', ' Never-worked', ' Private', ' Self-emp-inc', ' Self-emp-not-inc', ' State-gov', ' Without-pay'"


### Schema Environments

We need separate schema for training and serving environment, because the data in serving environment won't contain label and our schema should not raise an alarm when this happens. To avoid we will create environment wise schemas.

In [11]:
#Create schema environments
schema.default_environment.append('TRAINING')
schema.default_environment.append('SERVING')

#Remove label from serving env
tfdv.get_feature(schema, 'label').not_in_environment.append('SERVING')

In [12]:
_updated_schema_dir = f'{_pipeline_root}/updated_schema'

!mkdir -p {_updated_schema_dir}

schema_file = os.path.join(_updated_schema_dir, 'schema.pbtxt')
tfdv.write_schema_text(schema, schema_file)

### Import SchemaGen

In [13]:
user_schema_importer = tfx.components.ImportSchemaGen(schema_file = schema_file)
context.run(user_schema_importer, enable_cache = False)

ExecutionResult(
    component_id: ImportSchemaGen
    execution_id: 29
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=ImportSchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [14]:
context.show(user_schema_importer.outputs['schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'age',INT,required,,min: 17; max: 90
'capital-gain',INT,required,,-
'capital-loss',INT,required,,-
'education',STRING,required,,'education'
'education-num',INT,required,,-
'fnlwgt',INT,required,,-
'hours-per-week',INT,required,,-
'label',STRING,required,,'label'
'marital-status',STRING,required,,'marital-status'


,Values
Domain,
'education',"' 10th', ' 11th', ' 12th', ' 1st-4th', ' 5th-6th', ' 7th-8th', ' 9th', ' Assoc-acdm', ' Assoc-voc', ' Bachelors', ' Doctorate', ' HS-grad', ' Masters', ' Preschool', ' Prof-school', ' Some-college'"
'label',"' <=50K', ' >50K'"
'marital-status',"' Divorced', ' Married-AF-spouse', ' Married-civ-spouse', ' Married-spouse-absent', ' Never-married', ' Separated', ' Widowed'"
'native-country',"' ?', ' Cambodia', ' Canada', ' China', ' Columbia', ' Cuba', ' Dominican-Republic', ' Ecuador', ' El-Salvador', ' England', ' France', ' Germany', ' Greece', ' Guatemala', ' Haiti', ' Honduras', ' Hong', ' Hungary', ' India', ' Iran', ' Ireland', ' Italy', ' Jamaica', ' Japan', ' Laos', ' Mexico', ' Nicaragua', ' Outlying-US(Guam-USVI-etc)', ' Peru', ' Philippines', ' Poland', ' Portugal', ' Puerto-Rico', ' Scotland', ' South', ' Taiwan', ' Thailand', ' Trinadad&Tobago', ' United-States', ' Vietnam', ' Yugoslavia', ' Holand-Netherlands'"
'occupation',"' ?', ' Adm-clerical', ' Armed-Forces', ' Craft-repair', ' Exec-managerial', ' Farming-fishing', ' Handlers-cleaners', ' Machine-op-inspct', ' Other-service', ' Priv-house-serv', ' Prof-specialty', ' Protective-serv', ' Sales', ' Tech-support', ' Transport-moving'"
'race',"' Amer-Indian-Eskimo', ' Asian-Pac-Islander', ' Black', ' Other', ' White'"
'relationship',"' Husband', ' Not-in-family', ' Other-relative', ' Own-child', ' Unmarried', ' Wife'"
'sex',"' Female', ' Male'"
'workclass',"' ?', ' Federal-gov', ' Local-gov', ' Never-worked', ' Private', ' Self-emp-inc', ' Self-emp-not-inc', ' State-gov', ' Without-pay'"


### ExampleValidator

In [15]:
example_validator = tfx.components.ExampleValidator(
    statistics = statistics_gen.outputs['statistics'],
    schema = user_schema_importer.outputs['schema']
)

context.run(example_validator)

ExecutionResult(
    component_id: ExampleValidator
    execution_id: 30
    outputs:
        anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=ExampleValidator, output_key=anomalies, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [16]:
context.show(example_validator.outputs['anomalies'])

### ML Metadata

In [17]:
import ml_metadata as mlmd
from ml_metadata.proto import metadata_store_pb2

connection_config = context.metadata_connection_config
store = mlmd.MetadataStore(connection_config)

In [18]:
artifact_types = store.get_artifact_types()

[artifact_type.name for artifact_type in artifact_types]

['Examples',
 'ExampleStatistics',
 'Schema',
 'ExampleAnomalies',
 'TransformGraph',
 'TransformCache']

In [19]:
schema_list = store.get_artifacts_by_type('Schema')

[(f'schema uri: {schema.uri}', f'schema id:{schema.id}') for schema in schema_list]

[('schema uri: ./pipeline/SchemaGen/schema/5', 'schema id:5'),
 ('schema uri: ./pipeline/Transform/pre_transform_schema/7', 'schema id:10'),
 ('schema uri: ./pipeline/Transform/post_transform_schema/7', 'schema id:12'),
 ('schema uri: ./pipeline/SchemaGen/schema/10', 'schema id:17'),
 ('schema uri: ./pipeline/Transform/pre_transform_schema/12', 'schema id:22'),
 ('schema uri: ./pipeline/Transform/post_transform_schema/12', 'schema id:24'),
 ('schema uri: ./pipeline/SchemaGen/schema/18', 'schema id:32'),
 ('schema uri: ./pipeline/Transform/pre_transform_schema/20', 'schema id:37'),
 ('schema uri: ./pipeline/Transform/post_transform_schema/20', 'schema id:39'),
 ('schema uri: ./pipeline/SchemaGen/schema/23', 'schema id:44'),
 ('schema uri: ./pipeline/ImportSchemaGen/schema/24', 'schema id:45'),
 ('schema uri: ./pipeline/SchemaGen/schema/28', 'schema id:49'),
 ('schema uri: ./pipeline/ImportSchemaGen/schema/29', 'schema id:50')]

In [20]:
example_anomalies = store.get_artifacts_by_type('ExampleAnomalies')[0]

print(f'Artifact id: {example_anomalies.id}')

Artifact id: 6


In [21]:
anomalies_id_event = store.get_events_by_execution_ids([example_anomalies.id])[0]

print(anomalies_id_event)

artifact_id: 4
execution_id: 6
path {
  steps {
    key: "statistics"
  }
  steps {
    index: 0
  }
}
type: INPUT
milliseconds_since_epoch: 1689531925791



In [22]:
anomalies_execution_id = anomalies_id_event.execution_id
events_execution = store.get_events_by_execution_ids([anomalies_execution_id])
print(events_execution)

[artifact_id: 4
execution_id: 6
path {
  steps {
    key: "statistics"
  }
  steps {
    index: 0
  }
}
type: INPUT
milliseconds_since_epoch: 1689531925791
, artifact_id: 5
execution_id: 6
path {
  steps {
    key: "schema"
  }
  steps {
    index: 0
  }
}
type: INPUT
milliseconds_since_epoch: 1689531925791
, artifact_id: 6
execution_id: 6
path {
  steps {
    key: "anomalies"
  }
  steps {
    index: 0
  }
}
type: OUTPUT
milliseconds_since_epoch: 1689531925816
]
